In [170]:
import numpy as np
import pandas as pd

In [171]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [172]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA7/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [173]:
mineralogy = mineralogy.dropna()

In [174]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [175]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [176]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [177]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
6960,64.432487,0.735848,15.640608,1.437894,3.797282,0.134290,3.849654,4.345018,2.929597,2.682693,0.014628
6961,64.378915,0.739211,15.640489,1.440889,3.806367,0.134638,3.876309,4.363605,2.927251,2.677745,0.014580
6962,64.328612,0.742370,15.640327,1.443691,3.814875,0.134964,3.901406,4.381056,2.925048,2.673115,0.014535
6963,64.281679,0.745318,15.640131,1.446299,3.822794,0.135268,3.924883,4.397336,2.922993,2.668808,0.014493
6964,64.238210,0.748048,15.639912,1.448707,3.830112,0.135548,3.946678,4.412412,2.921088,2.664830,0.014454


In [187]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
6960,64.432487,0.735848,15.640608,1.437894,3.797282,0.134290,3.849654,4.345018,2.929597,2.682693,0.014628
6961,64.378915,0.739211,15.640489,1.440889,3.806367,0.134638,3.876309,4.363605,2.927251,2.677745,0.014580
6962,64.328612,0.742370,15.640327,1.443691,3.814875,0.134964,3.901406,4.381056,2.925048,2.673115,0.014535
6963,64.281679,0.745318,15.640131,1.446299,3.822794,0.135268,3.924883,4.397336,2.922993,2.668808,0.014493
6964,64.238210,0.748048,15.639912,1.448707,3.830112,0.135548,3.946678,4.412412,2.921088,2.664830,0.014454
...,...,...,...,...,...,...,...,...,...,...,...
262824,73.145524,0.222604,14.505840,0.799168,1.956296,0.065892,0.656030,1.411888,3.300728,3.905551,0.030479
262825,73.148133,0.222471,14.504952,0.798923,1.955625,0.065868,0.655457,1.411091,3.300834,3.906156,0.030489
262826,73.150946,0.222328,14.503994,0.798658,1.954902,0.065842,0.654840,1.410234,3.300948,3.906808,0.030500
262827,73.153949,0.222174,14.502971,0.798376,1.954129,0.065814,0.654181,1.409318,3.301070,3.907505,0.030512


In [184]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:160000]

In [188]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA7/area7_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA7/area7_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA7/area7_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA7/area7_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [189]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA7/area7_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA7/area7_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA7/area7_3_results_excel.txt")
df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA7/area7_4_results_excel.txt")

In [199]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [200]:
df_final

,Q,P,K
SAMPLE,,,
6960,19.920746,43.767987,18.331131
6961,19.815811,43.842640,18.299953
6962,19.717150,43.912780,18.270673
6963,19.635115,43.931264,18.243521
6964,19.561594,43.936475,18.218394
...,...,...,...
262824,33.855816,31.940018,25.871190
262825,33.858490,31.936745,25.874877
262826,33.861340,31.933328,25.878820


## QAPF classification

In [201]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [202]:
df_final["QAPF"].unique()

array(['granodiorite', 'monzo granite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [203]:
df_final.to_excel("../_CIPW/CIPW/AREA7/qapf.xlsx", index=True)

-------

In [204]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA7/QAPF_counts.xlsx")

-----

In [205]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area7.xlsx", index_col=None, usecols = ['area', 'Y_UTM', 'X_UTM', 'ZoneNumber', 'ZoneLetter'])

In [206]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,511633.869355,5.052403e+06,53.0,T,7.0,52.549507,17.912818,22.931202,monzo granite
2,535530.362355,5.123776e+06,53.0,T,7.0,45.940433,19.086384,32.357653,monzo granite
3,523597.931141,5.094252e+06,53.0,T,7.0,50.739986,6.408381,37.371890,syeno granite
4,529209.364967,5.079916e+06,53.0,T,7.0,41.407116,23.232361,27.590113,monzo granite
5,526764.683928,5.074365e+06,53.0,T,7.0,54.633030,17.945563,19.744849,monzo granite
...,...,...,...,...,...,...,...,...,...
98,531250.244842,5.036709e+06,53.0,T,7.0,38.970135,27.092730,30.356001,monzo granite
99,561486.936139,5.141199e+06,53.0,T,7.0,45.501648,14.674619,34.151529,syeno granite
100,563260.439707,5.114646e+06,53.0,T,7.0,39.089985,31.289243,26.202475,monzo granite
101,564967.071345,5.113837e+06,53.0,T,7.0,37.852432,28.802002,30.023175,monzo granite


In [207]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA7/QAPF_control_points.xlsx", index=True)